In [2]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

from torchnlp.datasets import multi30k_dataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train = multi30k_dataset(train=True)
validation = multi30k_dataset(dev=True)
test = multi30k_dataset(test=True)

## Loading data file
file format: 
[{'en': 'A man in an orange hat starring at something.', 
'de': 'Ein Mann mit einem orangefarbenen Hut, der etwas anstarrt.'}, {}]

1. add SOS EOS
2. From unicode to acsii, delete lower case and trim punctuation.

In [3]:
SOS_token = 0
EOS_token = 1

# build the dictionary
class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

def buildLangs(train, validation, test, reverse=False):
    """ build Language dictionary
    Input: set and reverse flag. Default file from Eng>De.
        If we want to translate De->Eng, set reverse=True
    Output: two language dictionary class from training set
    """
    train_pairs = [[normalizeString(pair['en']), normalizeString(pair['de'])] \
        for pair in train[0:1000]]
    # TODO: use all data

    if reverse:
        train_pairs = [list(reversed(pair)) for pair in train_pairs]
        input_lang = Lang('en')
        output_lang = Lang('de')
    else: 
        input_lang = Lang('de')
        output_lang = Lang('en')
    return input_lang, output_lang, train_pairs

def prepareData(reverse=False):
    input_lang, output_lang, pairs = buildLangs(train, validation, test, reverse)
    print("Read %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData(reverse=True)
print(random.choice(pairs))

Read 1000 sentence pairs
Counting words...
Counted words:
en 2176
de 1842
['ein mann mit einer blauen baseballkappe schlaft an einen felsen gelehnt .', 'a man with a blue baseball cap on is sleeping against a rock .']
